# Testing `Mistral-7B-v3`

We refer to the [Paper](https://arxiv.org/pdf/2310.06825).

## Importing libraries

We follow the [Documentation](https://qwenlm.github.io/blog/qwen1.5/).

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from huggingface_hub import login
import torch
import os
from pathlib import Path
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm

## Global variables

In [2]:
ROOT = Path("../../data")
subset_name = f"subset_{0}"
model_name = "gpt-4o-mini"
CHECKPOINT = ROOT / "cleaned_annotation" / f"{subset_name}_{model_name}_cleaned.parquet"

## If needed, use a proxy to connect to HuggingFace

If you're working in a restricted network environment and need access to Hugging Face models or other external services, you can route your Python or CLI traffic through a local SOCKS5 proxy using **Shadowsocks** (e.g., via Outline).

You can create a file : `start_proxy.sh`
```bash
#!/bin/bash

# Launch Shadowsocks local proxy connected to your Outline server

./sslocal \
  --server-addr YOUR_SERVER_ADDRESS \
  --encrypt-method chacha20-ietf-poly1305 \
  --password YOUR_KEY \
  --local-addr 127.0.0.1:1080 \
  -v
```

Then `chmod +x start_proxy.sh`.

And then run it in another terminal `./start_proxy.sh`. 

You can verify if it works with : `curl ipinfo.io --socks5-hostname 127.0.0.1:1080`.

In [3]:
os.environ["HTTP_PROXY"] = "socks5h://127.0.0.1:1080"
os.environ["HTTPS_PROXY"] = "socks5h://127.0.0.1:1080"

## Importing the model and the tokenizer 

Importing the model and tokenizer in the cache : `~/.cache/huggingface/hub/`

In [4]:
login(token=open("../../data/confidential/hf_token.txt").read().strip())

- Default loading in float32
- No memory optimization
- Tokenizer/model loaded to GPU without streaming

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,              # You can also try `load_in_8bit=True`
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

In [6]:
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"  # Automatically places model on GPU if available
)

num_params = sum(p.numel() for p in model.parameters())
print(f"Model: {model_name}")
print(f"Number of parameters: {num_params:,}")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model: mistralai/Mistral-7B-Instruct-v0.3
Number of parameters: 3,758,362,624


## Loading the data

In [7]:
df = pd.read_parquet(CHECKPOINT)

In [8]:
df

,msg_id,user,content,topic,deleted,banned,hour,explication,ton,intention,categorie,labels,toxicite_score,categorie_list,categorie_justification,labels_list,labels_justification,note,note_justification,certitude
0,anon_msg_9198a73c84ed,anon_user_079568ca56,osef le drapeau fr en allemagne a besoin de so...,anon_topic_de7347d6,1,1,5,Le message indique que le drapeau français en ...,"Tons identifiés : Apathique (80%), Critique (6...",1. Exprimer une opinion (degré de certitude : ...,"Catégories : [""Évocation ambiguë d’un propos t...",Labels : S0/H0/V0/R0/A2/I2 \nJustification : ...,Note : 2/10 \nJustification : Expression d'ap...,[Évocation ambiguë d’un propos toxique],Le message critique une situation sans être cl...,"[S0, H0, V0, R0, A2, I2]",Le message critique le manque de soutien sans ...,2/10,Expression d'apathie et critique légère sans a...,élevée
1,anon_msg_0ed0dcd30a95,anon_user_e8a049aa62,Ça rage sur les chad novax au sang 10/10,anon_topic_e697e741,1,0,10,Le message évoque une frustration ou une colèr...,"Tons identifiés : Indigné (80%), Moqueur (70%)...","Intentions identifiées : Critiquer (80%), Se m...","Catégories : [""Évocation d’un propos toxique a...",Labels : S0/H2/V0/R0/A2/I2 \nJustification : ...,Note : 4/10 \nJustification : Moquerie appuyé...,"[Évocation d’un propos toxique avec adhésion, ...","Le message se moque des ""chad novax"" en exprim...","[S0, H2, V0, R0, A2, I2]","Le message se moque d'un groupe spécifique, av...",4/10,Moquerie appuyée et condescendance envers un g...,élevée
2,anon_msg_b1641d66622a,anon_user_4fe7b2f2fe,"Pas moi, je ne connais pas de mégane",anon_topic_a64f2ab6,1,0,5,Le message exprime que l'auteur ne connaît pas...,"Tons identifiés : Neutre (certitude élevée), D...","Intentions identifiées : Informer, Exprimer un...",Catégories : []\nJustification : Le message es...,Labels : S0/H0/V0/R0/A0/I0 \nJustification : ...,"Note : 0/10 \nJustification : Message neutre,...",[],"Le message est neutre et factuel, sans indicat...","[S0, H0, V0, R0, A0, I0]","Le message est factuel et neutre, sans contenu...",0/10,"Message neutre, factuel et sans connotation ho...",élevée
3,anon_msg_8c19931d9a0b,anon_user_ab4654f2c9,l'été arrive et l'odeur des pieds puants des k...,anon_topic_e6ef1ef7,1,0,5,Le message évoque l'arrivée de l'été et décrit...,"Tons identifiés : Ironique (80%), Humoristique...",Les intentions identifiées sont : Exprimer une...,"Catégories : [""Humour à double lecture"", ""Évoc...",Labels : S0/H2/V0/R0/A2/I2 \nJustification : ...,Note : 4/10 \nJustification : Moquerie et mép...,"[Humour à double lecture, Évocation ambiguë d’...",Le message utilise l'humour pour critiquer un ...,"[S0, H2, V0, R0, A2, I2]","Pas d'allusion sexuelle, mais moquerie stéréot...",4/10,Moquerie et mépris stéréotypés avec intention ...,moyenne
4,anon_msg_80440252f1ca,anon_user_a034768ac3,Si tu es épanoui et que tu vis bien c'est très...,anon_topic_d956e559,1,1,5,Le message indique que si quelqu'un se sent ép...,Tons identifiés : Bienveillant (certitude élev...,"Intentions identifiées : Valoriser, Informer, ...",Catégories : []\nJustification : Le message es...,Labels : S0/H0/V0/R0/A0/I0 \nJustification : ...,Note : 0/10 \nJustification : Message bienvei...,[],Le message est clairement bienveillant et valo...,"[S0, H0, V0, R0, A0, I0]","Message bienveillant et factuel, sans allusion...",0/10,"Message bienveillant et positif, valorisant le...",élevée
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21399,anon_msg_f0994549b621,anon_user_d1536ba4ce,El famoso ovni aliens avec un parachute,anon_topic_8d425fc1,1,1,3,- **Sujet du message** : Mention d'un objet vo...,"Tons perçus : Neutre, Humoristique\n\nJustific...",1. **Informer** (certitude : élevée) \n Jus...,"Catégories : [""Humour à double lecture""]\nJust...",Labels : S0/H0/V0/R3/A0/I1 \nJustification : ...,Note : 1/10 \nJustification : Le message est ...,[Humour à double lecture],Le message utilise une combinaison absurde pou

## Generating a summary with `Mistral-7B-v3`

In [9]:
def prompt(row):
    return f"""
Tu es un annotateur expert en détection de contenu toxique. On considère un contenu est toxique s'il est vraiment indésirable sans exagérer.

On va te donner :
- un texte,
- un score de toxicité (sur une échelle de 0 à 10),
- une justification de ce score.

Ta tâche : évaluer si ce score est **approprié** par rapport au contenu.

Tu dois répondre **uniquement** par l’une des quatre réponses suivantes (sans rien ajouter d’autre) :
["Non, pas du tout", "Plutôt non", "Plutôt oui", "Oui, tout à fait"]

--- 

Voici la grille de notation utilisée pour le score de toxicité :

0. Neutre, informatif, bienveillant. Aucun jugement, connotation problématique ou sous-entendu offensif - Ex: « Je déteste la SF »
1. Ton léger ou familier, humour inoffensif, remarque banale ou ironie douce - Ex: « Quel boulet »
2. Vulgarité légère, stéréotype bénin ou jugement anodin. Peu susceptible d’être perçu comme hostile - Ex: « Les boomers comprennent rien à TikTok »
3. Stéréotype implicite, ton ambigu pouvant heurter certaines sensibilités sans attaque claire - Ex: « Les Parisiens, toujours à râler »
4. Moquerie appuyée, condescendance, biais culturel, potentiellement blessant mais sans appel à la haine - Ex: « Tu t’exprimes bien pour un rebeu »
5. Préjugé, propos problématique ou stigmatisant, controverses morales, sans incitation à la haine - Ex:  « Les trans, c’est quand même un délire »
6. Dévalorisation claire, propos dégradants ou intolérables, humiliation, avec une intention nuisible - Ex: « T’es qu’une sous-merde » 
7. Violence verbale forte, sexualisation dégradante, menace, prise de position très problématique, franchit le seuil d’hostilité directe - Ex: « Les trans, c’est une pathologie mentale »
8. Discours de haine, banalisations de sujets graves, etc, idéologie ou agressivité clairement toxique - Ex: « Un bon flic est un flic mort »
9. Incitation claire à la haine, à la violence ou au suicide, propos gravement menaçants, choquants ou harcèlement manifeste - Ex: « Va te suicider, personne ne t’aime »
10. Propos inhumains, illégaux ou moralement condamnables, avec adhésion - Ex: « J’ai violé une gamine »

---

Texte :
{row['content']}

Score de toxicité proposé :
{row['note']}

Justification proposée :
{row['note_justification']}

---

Le score est-il approprié ? Réponds uniquement par l’une des quatre réponses possibles :
- Non, pas du tout
- Plutôt non
- Plutôt oui
- Oui, tout à fait

Réponse :
"""

In [10]:
def generate_batch(df_batch, tokenizer, model, device, max_new_tokens=10):
    prompts = [prompt(row) for _, row in df_batch.iterrows()]
    tokens = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(device)
    pad_token_id = tokenizer.pad_token_id or tokenizer.eos_token_id

    with torch.no_grad():
        outputs = model.generate(
            **tokens,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=pad_token_id
        )
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return [r.split("Réponse :")[-1].strip() for r in decoded]

In [11]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [12]:
batch_size = 64  
results = []

for i in tqdm(range(0, len(df), batch_size), desc="Batch inference"):
    batch_df = df.iloc[i:i+batch_size]
    batch_results = generate_batch(batch_df, tokenizer, model, device)
    results.extend(batch_results)

df["annotation"] = results


Batch inference:   0%|          | 0/335 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Batch inference: 100%|██████████| 335/335 [1:50:35<00:00, 19.81s/it]


In [13]:
df

,msg_id,user,content,topic,deleted,banned,hour,explication,ton,intention,...,labels,toxicite_score,categorie_list,categorie_justification,labels_list,labels_justification,note,note_justification,certitude,annotation
0,anon_msg_9198a73c84ed,anon_user_079568ca56,osef le drapeau fr en allemagne a besoin de so...,anon_topic_de7347d6,1,1,5,Le message indique que le drapeau français en ...,"Tons identifiés : Apathique (80%), Critique (6...",1. Exprimer une opinion (degré de certitude : ...,...,Labels : S0/H0/V0/R0/A2/I2 \nJustification : ...,Note : 2/10 \nJustification : Expression d'ap...,[Évocation ambiguë d’un propos toxique],Le message critique une situation sans être cl...,"[S0, H0, V0, R0, A2, I2]",Le message critique le manque de soutien sans ...,2/10,Expression d'apathie et critique légère sans a...,élevée,Plutôt oui
1,anon_msg_0ed0dcd30a95,anon_user_e8a049aa62,Ça rage sur les chad novax au sang 10/10,anon_topic_e697e741,1,0,10,Le message évoque une frustration ou une colèr...,"Tons identifiés : Indigné (80%), Moqueur (70%)...","Intentions identifiées : Critiquer (80%), Se m...",...,Labels : S0/H2/V0/R0/A2/I2 \nJustification : ...,Note : 4/10 \nJustification : Moquerie appuyé...,"[Évocation d’un propos toxique avec adhésion, ...","Le message se moque des ""chad novax"" en exprim...","[S0, H2, V0, R0, A2, I2]","Le message se moque d'un groupe spécifique, av...",4/10,Moquerie appuyée et condescendance envers un g...,élevée,Plutôt oui
2,anon_msg_b1641d66622a,anon_user_4fe7b2f2fe,"Pas moi, je ne connais pas de mégane",anon_topic_a64f2ab6,1,0,5,Le message exprime que l'auteur ne connaît pas...,"Tons identifiés : Neutre (certitude élevée), D...","Intentions identifiées : Informer, Exprimer un...",...,Labels : S0/H0/V0/R0/A0/I0 \nJustification : ...,"Note : 0/10 \nJustification : Message neutre,...",[],"Le message est neutre et factuel, sans indicat...","[S0, H0, V0, R0, A0, I0]","Le message est factuel et neutre, sans contenu...",0/10,"Message neutre, factuel et sans connotation ho...",élevée,"Oui, tout à fait"
3,anon_msg_8c19931d9a0b,anon_user_ab4654f2c9,l'été arrive et l'odeur des pieds puants des k...,anon_topic_e6ef1ef7,1,0,5,Le message évoque l'arrivée de l'été et décrit...,"Tons identifiés : Ironique (80%), Humoristique...",Les intentions identifiées sont : Exprimer une...,...,Labels : S0/H2/V0/R0/A2/I2 \nJustification : ...,Note : 4/10 \nJustification : Moquerie et mép...,"[Humour à double lecture, Évocation ambiguë d’...",Le message utilise l'humour pour critiquer un ...,"[S0, H2, V0, R0, A2, I2]","Pas d'allusion sexuelle, mais moquerie stéréot...",4/10,Moquerie et mépris stéréotypés avec intention ...,moyenne,Plutôt oui
4,anon_msg_80440252f1ca,anon_user_a034768ac3,Si tu es épanoui et que tu vis bien c'est très...,anon_topic_d956e559,1,1,5,Le message indique que si quelqu'un se sent ép...,Tons identifiés : Bienveillant (certitude élev...,"Intentions identifiées : Valoriser, Informer, ...",...,Labels : S0/H0/V0/R0/A0/I0 \nJustification : ...,Note : 0/10 \nJustification : Message bienvei...,[],Le message est clairement bienveillant et valo...,"[S0, H0, V0, R0, A0, I0]","Message bienveillant et factuel, sans allusion...",0/10,"Message bienveillant et positif, valorisant le...",élevée,"Oui, tout à fait"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21399,anon_msg_f0994549b621,anon_user_d1536ba4ce,El famoso ovni aliens avec un parachute,anon_topic_8d425fc1,1,1,3,- **Sujet du message** : Mention d'un objet vo...,"Tons perçus : Neutre, Humoristique\n\nJustific...",1. **Informer** (certitude : élevée) \n Jus...,...,Labels : S0/H0/V0/R3/A0/I1 \nJustification : ...,Note : 1/10 \nJustification : Le message est ...,[Humour à double lecture],Le message utilise une combinaison absurde pou...,"[S0, H0, V0, R3, A0, I1]",Le message est humoristique avec une référence...,1/10,"Le message est humoristique, neutre, sans cont...",élevée,Plutôt oui
21400,anon_msg_4c3a0fb9028d,anon_user_d0fa84cce4,"""Jud

In [16]:
df.to_parquet(CHECKPOINT.with_name(f"{CHECKPOINT.stem}_mistral.parquet"))